<a href="https://colab.research.google.com/github/KzKe/Transformer-models/blob/master/Models/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding	
Reference:
1. [The Annotated Transformer from HarvardNLP](https://nlp.seas.harvard.edu/2018/04/03/attention.html)
2. [Attention is all you need](https://arxiv.org/abs/1706.03762)

# 背景


#模型

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

# 训练

# 示例